# CO2 Emissions Dashboard

## Introduction 
In recent years, concern over how greenhouse gas emissions, particularly **carbon dioxide (CO2)**, affect the temperature of the planet has grown significantly. Many human activities, such as the combustion of fossil fuels, deforestation, and industrial operations, all result in CO2 emissions. Creating successful climate policies and reducing the effects of climate change depend on knowing how much CO2 each nation emits.

## Dataset
The dataset on CO2 emissions offers a thorough summary of the total quantity of CO2 emitted by every nation. The dataset contains data on CO2 emissions by nation between 1960 to the present. It includes all nations and is compiled from a number of sources, including the International Energy Agency and the United Nations Framework Convention on Climate Change (UNFCCC) (IEA).

Researchers, decision-makers, and the general public can use the dataset to acquire understanding of the relative contributions of various nations to global CO2 emissions. Additionally, it can be used to evaluate the success of climate policy and track changes in emissions over time.

The CO2 emissions dataset is a crucial tool for comprehending the global climate crisis and for creating mitigation plans for CO2 emissions' negative effects on the environment.

## Import Packages 

In [3]:
import panel as pn
import pandas as pd
import param
import hvplot.pandas

from pycountry_convert import country_name_to_country_alpha2, country_alpha2_to_continent_code

## Load Dataset

In [4]:
CO2_df = pd.read_csv('data/co2_emissions_kt_by_country.csv')

In [5]:
CO2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13953 entries, 0 to 13952
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  13953 non-null  object 
 1   country_name  13953 non-null  object 
 2   year          13953 non-null  int64  
 3   value         13953 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 436.2+ KB


In [6]:
CO2_df.head()

,country_code,country_name,year,value
0,ABW,Aruba,1960,11092.675
1,ABW,Aruba,1961,11576.719
2,ABW,Aruba,1962,12713.489
3,ABW,Aruba,1963,12178.107
4,ABW,Aruba,1964,11840.743


In [7]:
CO2_df.describe()

,year,value
count,13953.000000,1.395300e+04
mean,1990.732316,8.254983e+05
std,17.187585,2.788923e+06
min,1960.000000,-8.067400e+01
25%,1976.000000,1.100000e+03
50%,1992.000000,1.390000e+04
75%,2006.000000,1.642779e+05
max,2019.000000,3.434401e+07


## Preprocessing

### Add Continent Column

In [8]:
def add_continent(country_name):
    """
    Returns the continent for a given country name using pycountry_convert library.

    Args:
        country_name (str): Name of the country.

    Returns:
        str: Continent name.
    """
    try:
        country_alpha2 = country_name_to_country_alpha2(country_name)
        continent_code = country_alpha2_to_continent_code(country_alpha2)
        continent_names = {
            'AF': 'Africa',
            'AN': 'Antarctica',
            'AS': 'Asia',
            'EU': 'Europe',
            'NA': 'North America',
            'OC': 'Oceania',
            'SA': 'South America'
        }
        continent_name = continent_names.get(continent_code)
        return continent_name
    except Exception as e:
        print("Error: ", str(e))
        return None


In [ ]:
CO2_df["continent"] = CO2_df['country_name'].apply(add_continent)

In [10]:
CO2_df.head()

,country_code,country_name,year,value,continent
0,ABW,Aruba,1960,11092.675,North America
1,ABW,Aruba,1961,11576.719,North America
2,ABW,Aruba,1962,12713.489,North America
3,ABW,Aruba,1963,12178.107,North America
4,ABW,Aruba,1964,11840.743,North America


In [11]:
CO2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13953 entries, 0 to 13952
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  13953 non-null  object 
 1   country_name  13953 non-null  object 
 2   year          13953 non-null  int64  
 3   value         13953 non-null  float64
 4   continent     10075 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 545.2+ KB


## Drop rows with null vallues

In [12]:
CO2_df = CO2_df.dropna()

In [13]:
CO2_df.isnull().sum()

country_code    0
country_name    0
year            0
value           0
continent       0
dtype: int64

## Build Dashboard

In [92]:
class CO2Emmisions(param.Parameterized):
    
    continent     = param.ObjectSelector()
    country       = param.ObjectSelector()
    year          = param.Integer(2000, bounds=(1960, 2019))
    country_table = param.Parameter()
    co2_df        = param.DataFrame()
    
    #constructor
    def __init__(self, co2_data, **params):
        super().__init__(**params)
        self.co2_df = co2_data
        
        self.country_table = pn.widgets.Tabulator( pagination='remote', page_size = 10, 
                                             show_index=False, hidden_columns =['year', 'continent'])
        
        self.get_continent()
    
    
    def get_continent(self):
    '''
    Retrieves all the continent names in the dataset into a list and 
    then populates the continent selector object
    ''' 
        try:
            continents = self.co2_df['continent'].drop_duplicates()
            self.param['continent'].objects = sorted(continents.to_list())
            self.continent = continents[0]
            
        except Exception as e:
            print("Error: ", str(e))

    @param.depends('continent', watch=True)
    def get_countries(self):
        """
        Retrieves the countries that belong to the continent selected
        then populates the country selector object.

        """
        try:
            countries = self.co2_df[self.co2_df['continent'] == self.continent]['country_name'].drop_duplicates().tolist()
            self.param['country'].objects = countries
            self.country = countries[0]
        
        except Exception as e:
            print("Error: ", str(e))
            return None
        
    @param.depends('year')
    def co2_continent_plot(self):
    '''
    Returns a line plot of the CO2 level vs Year by country
    
    return: hvplot line plot 
    ''' 
        co2_continent = (
            self.co2_df[
                (self.co2_df.year <= self.year)
            ]
            .groupby(['continent', 'year'])['value'].mean()
            .reset_index()
            .sort_values(by='year')  
            .reset_index(drop=True)
        )

        return co2_continent.hvplot(x='year', by='continent', y='value', line_width=2, height=400, width=650, yaxis=1**6, title="CO2 emission by Continent")
        
    
    def co2_country_table(self):
    '''
    Returns a table with a filtered list by continent and year of the country, CO2 levels, year 
    
    return: pandas table
    '''   
        try:
            co2_country = (
                self.co2_df[
                    (self.co2_df.year <= self.year) & (self.co2_df.continent == self.continent)
                ]
                .reset_index()
                .sort_values(by='year')  
                .reset_index(drop=True)
            )
            
            self.country_table.value = co2_country
        
        except Exception as e:
            print("Error: ", str(e))
            return None
    
    def co2_country_plot(self):
        try:
            co2_country = (
                self.co2_df[
                    (self.co2_df.year <= self.year) & (self.co2_df.continent == self.continent)
                ]
                .reset_index()
                .sort_values(by='year')  
                .reset_index(drop=True)
            )
            
            return co2_country.hvplot(x = 'year', by='country_name', y='value',line_width=2, height=400, width=650, title="CO2 emission by Country")
        
        except Exception as e:
            print("Error: ", str(e))
            return None
        


    @param.depends('country')
    def view_map(self):
        iframe = """
        <iframe width="600" height="300" src="https://maps.google.com/maps?q={country}&z=6&output=embed"
        frameborder="0" scrolling="no" marginheight="0" marginwidth="0"></iframe>
        """.format(country=self.country)
        return pn.pane.HTML(iframe, height=400)
    
    @param.depends('continent','country', 'year', watch=True)
    def update(self):
        self.co2_country_table()
        self.co2_country_plot()
        

## Use Panel FastListTemplate to create Dashboard

In [ ]:
co2EmmisionsViewer = CO2Emmisions(CO2_df)

co2_dashboard = pn.template.FastListTemplate(
    title="CO2 Emissions",
    sidebar=[pn.pane.Markdown("# CO2 Emissions and Climate Change"), 
             pn.pane.Markdown("#### Carbon dioxide emissions are the primary driver of global climate change."), 
             pn.pane.Markdown("## Settings"),   
             co2EmmisionsViewer.param.continent,
             co2EmmisionsViewer.param.country,
             co2EmmisionsViewer.param.year],
    main=[
        pn.Row(
            pn.Column(
                co2EmmisionsViewer.co2_country_plot,
                '#### CO2 emission by Country',
                co2EmmisionsViewer.country_table,
            ),
            pn.Column(
                co2EmmisionsViewer.co2_continent_plot,
                '#### Location View',
                co2EmmisionsViewer.view_map,
            )
        )
    ],
).show();